In [16]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings

# Configuration of display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', None)
sns.set_style(style='whitegrid')
warnings.filterwarnings('ignore')

%matplotlib inline
print('Libraries Imported Successfully')

Libraries Imported Successfully


## Data Exploration

In [17]:
data = pd.read_csv('/kaggle/input/bank-customer-churn/botswana_bank_customer_churn.csv')

In [18]:
print(f'{data.shape} \n')
print(f'{data.info()} \n')

(115640, 25) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115640 entries, 0 to 115639
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   RowNumber                        115640 non-null  int64  
 1   CustomerId                       115640 non-null  object 
 2   Surname                          115640 non-null  object 
 3   First Name                       115640 non-null  object 
 4   Date of Birth                    115640 non-null  object 
 5   Gender                           115640 non-null  object 
 6   Marital Status                   115640 non-null  object 
 7   Number of Dependents             115640 non-null  int64  
 8   Occupation                       115640 non-null  object 
 9   Income                           115640 non-null  float64
 10  Education Level                  115640 non-null  object 
 11  Address                          115640 non-null  

In [22]:
print(f"Total unique values for CustomerId are {data['CustomerId'].nunique()}")
print(f"The unique number of occupation we have here are {data['Occupation'].nunique()}")
print(f"""The min date for a churn customer is {data['Churn Date'].min()} 
The max date for a churn customer is {data['Churn Date'].max()}""")

Total unique values for CustomerId are 115640
The unique number of occupation we have here are 639
The min date for a churn customer is 2024-01-01 00:00:00 
The max date for a churn customer is 2024-08-07 00:00:00


In [23]:
data.head(5)

,RowNumber,CustomerId,Surname,First Name,Date of Birth,Gender,Marital Status,Number of Dependents,Occupation,Income,Education Level,Address,Contact Information,Customer Tenure,Customer Segment,Preferred Communication Channel,Credit Score,Credit History Length,Outstanding Loans,Churn Flag,Churn Reason,Churn Date,Balance,NumOfProducts,NumComplaints
0,1,83ef0b54-35f6-4f84-af58-5653ac0c0dc4,Smith,Troy,1987-08-29,Male,Divorced,3,Information systems manager,77710.14,High School,"26644 Diaz Viaduct\nNorth Jessicaville, VI 14852",001-301-934-3058x0825,30,Retail,Phone,397,24,41959.74,0,NaN,NaT,211359.05,1,0
1,2,009f115a-e5ca-4cf4-97d6-530140545e4e,Sullivan,Katrina,2000-02-07,Female,Married,1,Charity fundraiser,58209.87,High School,"5642 Rachel Pass Suite 320\nPort Peggy, RI 75805",+1-581-683-4267,27,SME,Email,665,10,8916.67,0,NaN,NaT,30624.76,4,1
2,3,66309fd3-5009-44d3-a3f7-1657c869d573,Fuller,Henry,1954-02-03,Female,Single,1,Television production assistant,9794.01,High School,"00025 Brittany Flats Apt. 543\nNew Amandaborough, MH 69279",9753900004,14,Retail,Email,715,21,43270.54,0,NaN,NaT,111956.61,2,6
3,4,b02a30df-1a5f-4087-8075-2a35432da641,Young,Antonio,1991-01-15,Female,Divorced,5,Agricultural engineer,15088.98,High School,Unit 3386 Box 0088\nDPO AE 71940,+1-928-477-2856x660,23,Corporate,Phone,747,17,17887.65,0,NaN,NaT,201187.61,1,0
4,5,0d932e5b-bb3a-4104-8c83-f84270f7f2ea,Andersen,John,1992-04-08,Female,Divorced,2,"Teacher, early years/pre",60726.56,Master's,"0120 Leslie River\nWest Lindseychester, MN 98290",204.515.2388,22,Corporate,Email,549,25,32686.84,0,NaN,NaT,60391.24,5,6


#### **Data Review**
1. CustomerId is the Primary key. All 115640 values are unique, there is no repetition in the data and no problem in model fitting. As these are all individual customers.
2. RowNumber, CustomerId, Surname, FirstName, Address (No region wise segmentation) , Contact Infromation are no use for analysis of churn.
3. Date of Birth and Churn Date column are not a datetime column. DOB will be used for age calculation, churn for comparing.
4. The column to be used in Churn analysis are - Date of Birth, Gender, Martial Status, Number of Dependents, occupation, Income, Education level, Customer Tenure, Preferred Communication Channel, Credit Score, Credit history Length, outstanding loans.
5. Assuming Balance, NumOfProducts and NumComplaints are also contributing factor of churning.

#### **Assumption** 
1. NumComplaints, NumOfProducts, Balance are counted before the customer churned and influence the customer churn decision.
2. The NaN and NaT represents that the customer has not churned yet, this is also validated by the '0' Value in the Churn Flag.
3. Considering Customer Tenure as Months. As the legal age of opening bank accounts is 10+ Years of age. An example where person born in 1996 and churned in 2024 have the tenure to be 23. If it is year then the person must have opened the account at 5 year old of age which doesn't make sense.

In [24]:
data['Date of Birth'] = pd.to_datetime(data['Date of Birth'], format="mixed")
print(f"The column Date of Birth has the correct format which is {data['Date of Birth'].dtype}.")
data['Churn Date'] = pd.to_datetime(data['Churn Date'], format="mixed")
print(f"The column Date of Birth has the correct format which is {data['Churn Date'].dtype}.")

The column Date of Birth has the correct format which is datetime64[ns].
The column Date of Birth has the correct format which is datetime64[ns].


In [25]:
print(f"The number of people that have churned out of 115640 are {116500 - data['Churn Reason'].isnull().sum()}")
print("Rest all the columns in the dataset are of correct datatype as they should be.")

The number of people that have churned out of 115640 are 14954
Rest all the columns in the dataset are of correct datatype as they should be.


In [26]:
data[data['Churn Flag'] == 1].head()

,RowNumber,CustomerId,Surname,First Name,Date of Birth,Gender,Marital Status,Number of Dependents,Occupation,Income,Education Level,Address,Contact Information,Customer Tenure,Customer Segment,Preferred Communication Channel,Credit Score,Credit History Length,Outstanding Loans,Churn Flag,Churn Reason,Churn Date,Balance,NumOfProducts,NumComplaints
5,6,c18b027e-b108-4900-b413-dac87e2ee9b5,Gonzales,Ashley,1987-11-28,Male,Single,0,"Surveyor, building",55608.75,Bachelor's,"PSC 3787, Box 9869\nAPO AP 62324",001-880-967-5883x0508,15,Retail,Email,809,27,37218.81,1,Account Closure,2024-07-31,31206.81,2,10
16,17,963aaa61-c10f-46cb-be53-51505d395bd1,Medina,Adam,1996-10-24,Male,Divorced,2,Animal nutritionist,19725.99,Master's,"5990 Douglas Street Apt. 578\nRobertsonmouth, ID 08914",544.773.2718x2419,23,Retail,Email,399,22,31765.09,1,Relocation,2024-04-30,18249.32,4,10
18,19,b307039e-5723-4cfe-b793-cdb0822a66d9,Garcia,Mandy,1976-10-26,Female,Married,1,Civil Service administrator,17002.05,Diploma,"3267 Haynes Roads Apt. 605\nKnightview, MD 25372",747-623-3046,23,Retail,Email,516,30,37680.08,1,Account Closure,2024-05-05,47496.00,1,6
21,22,a654eb64-9cce-4cd7-835a-5c9a1d73cb33,Martinez,Kevin,1990-01-15,Male,Single,0,"Engineer, civil (contracting)",39981.11,Bachelor's,Unit 9654 Box 7101\nDPO AE 25766,2685896429,25,Retail,Phone,437,2,6761.18,1,Account Closure,2024-06-13,75231.59,2,9
36,37,344350d9-0447-4ac3-bb07-a5f9f6fa0529,Maxwell,James,1996-05-30,Male,Single,5,Podiatrist,50627.29,Bachelor's,"71357 Robert Skyway Apt. 019\nEast David, IL 08628",(755)273-7818x83131,8,Corporate,Phone,459,6,6271.79,1,Account Closure,2024-03-06,40373.67,3,10


#### May be the addressed can be used for analysis
The address can be decoded with some abbrevations repeating before the pin code. 

* VI (Virgin Islands)
* RI (Rhode Island)
* AE (Armed Forces Europe)


In [27]:
data['States'] = data['Address'].str.extract(r',\s([A-Z]{2})\s\d{5}')
data['States'] = data['States'].str.strip().str.upper()
data['States'].nunique()

59

In [28]:
region_map = {

    # Northeast
    'ME':'Northeast','NH':'Northeast','VT':'Northeast','MA':'Northeast',
    'RI':'Northeast','CT':'Northeast','NY':'Northeast','NJ':'Northeast','PA':'Northeast',

    # Midwest
    'OH':'Midwest','MI':'Midwest','IN':'Midwest','IL':'Midwest',
    'WI':'Midwest','MN':'Midwest','IA':'Midwest','MO':'Midwest',
    'ND':'Midwest','SD':'Midwest','NE':'Midwest','KS':'Midwest',

    # South
    'DE':'South','MD':'South','DC':'South','VA':'South','WV':'South',
    'NC':'South','SC':'South','GA':'South','FL':'South','KY':'South',
    'TN':'South','MS':'South','AL':'South','OK':'South','TX':'South',
    'AR':'South','LA':'South',

    # West
    'MT':'West','WY':'West','CO':'West','NM':'West','ID':'West',
    'UT':'West','AZ':'West','NV':'West','CA':'West','OR':'West',
    'WA':'West','AK':'West','HI':'West'
}

territories = ['PR','GU','AS','VI','FM','MH','MP','PW','AE','AP']

for t in territories:
    region_map[t] = 'Territory'

In [29]:
data['Region'] = data['States'].map(region_map)
data['Region'].value_counts()

Region
South        29733
West         22743
Midwest      21010
Northeast    15743
Territory    14041
Name: count, dtype: int64

In [30]:
print(data[data['Region'].isna()]['States'].unique())

[nan]


In [31]:
data[['States', 'Region']].head(10)

,States,Region
0,VI,Territory
1,RI,Northeast
2,MH,Territory
3,NaN,NaN
4,MN,Midwest
5,NaN,NaN
6,MP,Territory
7,PA,Northeast
8,NH,Northeast
9,TX,South
